from scrapping.images import get_images
from scrapping.box import get_box_labels

get_images()

get_box_labels()

In [ ]:
from PIL import Image
import numpy as np
image_path = "data/cropped/zackrawrr/cropped_image_5.png"
image = Image.open(image_path)
image_array = np.array(image)

In [ ]:
import pickle
with open("data/box/zackrawrr/cropped_image_5.pkl", "rb") as box_dictionary_labels:
    labels = pickle.load(box_dictionary_labels)

In [ ]:
#me fijo si alguno de los 2 bottom corners esta pegado a alguno de los 2 top corner del siguiente
#comparas los ultimos 2 con los primeros 2 del siguiente

#tien que ser alguno de los 2 

In [ ]:
#primero compara horizontalmente

#comparo top izquierdo con top derecho

In [ ]:
saved_index = []
for result_index in range(len(labels)-1):
    if (np.absolute(labels[result_index + 1][0][0][0] - labels[result_index][0][1][0])) < 30:
        saved_index.append(result_index)

In [ ]:
for index in saved_index:
    upper_y_first = labels[index][0][0][1] 
    upper_y_second = labels[index + 1][0][0][1]
    lower_y_first = labels[index][0][2][1]
    lower_y_second = labels[index + 1][0][2][1]    
    
    labels[index][0][1] = labels[index + 1][0][1]
    labels[index][0][2] = labels[index + 1][0][2]  

    labels[index][0][0][1] = max(upper_y_first,upper_y_second)
    labels[index][0][1][1] = max(upper_y_first,upper_y_second)
    labels[index][0][2][1] = min(lower_y_first,lower_y_second)
    labels[index][0][3][1] = min(lower_y_first,lower_y_second)

    mutable_list = list(labels[index])
    new_string = labels[index][1] + " " + labels[index + 1][1]    
    mutable_list[1] = new_string
    labels[index] = mutable_list
    
for delete_index in sorted(saved_index, reverse=True):
    del labels[delete_index + 1]

In [ ]:
#los puntos de la izquierda del index pasan a ser los puntos de la izquierda del index del siguiente. el texo pasa a ser la conbinacion de los 2

#los puntos de abajo sean iguales a los de abajo del proximo

In [ ]:
saved_index_0 = []
for result_index in range(len(labels)-1):
    if (np.absolute(labels[result_index + 1][0][0][1] - labels[result_index][0][2][1])) < 10:
        saved_index_0.append(result_index)   

In [ ]:
sequences = []
current_sequence = []

for i in range(len(saved_index_0)):
    if i == 0:
        current_sequence.append(saved_index_0[i])
    elif saved_index_0[i] == saved_index_0[i - 1] + 1:
        current_sequence.append(saved_index_0[i])
    else:
        sequences.append(current_sequence)
        current_sequence = [saved_index_0[i]]

# Append the last sequence
sequences.append(current_sequence)

In [ ]:
for index_list in sequences:

    min_left = 10000
    max_right = -10000
    
    for box_index in index_list:        
        if labels[box_index][0][0][0] < min_left:
            min_left = labels[box_index][0][0][0]
        if labels[box_index][0][0][0] > max_right:
            max_right = labels[box_index][0][1][0]
    if labels[box_index][0][0][0] < min_left:
        min_left = labels[box_index[-1] + 1][0][0][0]
    if labels[box_index][0][0][0] > max_right:
        max_right = labels[box_index[-1] + 1][0][1][0]

   


    #labels[index_list[0]][0][0][0] = min_left
    #labels[index_list[0]][0][1][0] = max_right
    #labels[index_list[0]][0][2][0] = max_right
    #labels[index_list[0]][0][3][0] = min_left
    
    labels[index_list[0]][0][2][1] = labels[index_list[-1] + 1][0][2][1]
    labels[index_list[0]][0][3][1] = labels[index_list[-1] + 1][0][3][1]

    new_string = ""
    for box_index in index_list:           
        new_string = new_string + " " + labels[box_index][1] 
    new_string = new_string + " " + labels[index_list[-1] + 1][1]

    mutable_list = list(labels[index_list[0]])    
    mutable_list[1] = new_string
    labels[index_list[0]] = mutable_list

for delete_index_list in sorted(sequences, reverse=True):  
    for delete_index in delete_index_list[::-1]:
        del labels[delete_index + 1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import cv2
import numpy as np

In [ ]:

img = cv2.imread(image_path)

# Draw bounding boxes and text
for box, text, _ in labels:
    # Convert box to numpy array and ensure integer type
    pts = np.array(box, np.int32)
    pts = pts.reshape((-1, 1, 2))

    # Draw bounding box
    cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Add text
    org = (int(box[0][0]), int(box[0][1]) - 5)  # Ensure coordinates are integers
    cv2.putText(img, text, org, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Save the image with bounding boxes and text
cv2.imwrite("image_with_boxes_and_text.jpg", img)